# 3.5 Enrich Startup 

Clean and extract information from 3.3 enriching session

In [11]:
import pandas as pd

In [12]:
df_listings = pd.read_csv("./data/eustartup_listings_enriched_4.csv")

In [13]:
# show *all* rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the “page” so it won’t wrap columns  
pd.set_option('display.width', 1000)

In [14]:
df_listings.head(3)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons,startup,success_count,total_funding_eur,acquisition_count,bankruptcy_count,non_success_count
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,[],NaN,Avdain,0,0,0,0,0
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,https://x.com/FranzHoegl/status/18789481448322...,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l...",4,"['Insolvenz | Popper Power GmbH', 'Insolvenz |...","['Reports a risk event (insolvency).', 'Report...",Popper Power,0,0,0,4,0
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,https://www.linkedin.com/company/setterai?trk=...,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...,0,[],NaN,Setter AI,0,0,0,0,0


## Linked In Links

In [15]:

# 1) Build a boolean mask: non-null but not matching the AT company pattern
mask_invalid = (
    df_listings["LinkedIn"].notna()
    & ~df_listings["LinkedIn"].str.match(r"^https?://at\.linkedin\.com/company/.*", na=False)
)

# 2) Slice those rows
invalid_li = df_listings.loc[mask_invalid, ["name_cleaned", "LinkedIn"]]

# 3) Display them so you can review
print(f"{len(invalid_li)} invalid LinkedIn entries found:")
print(invalid_li.to_string(index=False))

95 invalid LinkedIn entries found:
             name_cleaned                                                                                                                LinkedIn
                Setter AI                       https://www.linkedin.com/company/setterai?trk=public_post_main-feed-card_reshare_feed-actor-image
             SurveySensum                                                                           https://www.linkedin.com/company/surveysensum
     Hyrox Training Plans                                                                              https://uk.linkedin.com/company/mycoachapp
                 Finslice                                                                https://www.linkedin.com/company/finslice?trk=ppro_cprof
                  CaterMe                                                                               https://www.linkedin.com/company/cater-me
                 RTOPilot                                                                

In [16]:
import pandas as pd

# 1) Define your global LinkedIn allow-list (normalized, no trailing slash)
global_profiles = [
    "https://www.linkedin.com/company/setterai",
    "https://www.linkedin.com/company/surveysensum",
    "https://www.linkedin.com/company/finslice",
    "https://www.linkedin.com/company/dignaai",
    "https://www.linkedin.com/company/money-care",
    "https://www.linkedin.com/company/easy-month-end",
    "https://www.linkedin.com/company/redeem-solar-technologies",
    "https://de.linkedin.com/company/simplexx-gmbh",
    "https://www.linkedin.com/company/afreshed",
    "https://www.linkedin.com/company/gekko-it-solutions-gmbh",
    "https://www.linkedin.com/company/arkeon-bio",
    "https://www.linkedin.com/company/leaders21com",
    "https://www.linkedin.com/company/propup-at",
    "https://www.linkedin.com/company/truendo",
    "https://www.linkedin.com/company/blackshark-ai",
    "https://www.linkedin.com/company/drainbot",
    "https://www.linkedin.com/company/sanusx/about?viewAsMember=true",
    "https://www.linkedin.com/company/kickscale",
    "https://www.linkedin.com/company/flight-levels",
    "https://www.linkedin.com/company/codeversity",
    "https://www.linkedin.com/company/imagebiopsy-lab",
    "https://de.linkedin.com/company/lama-app",
    "https://www.linkedin.com/company/mostlyai",
    "https://www.linkedin.com/company/contextflow",
    "https://www.linkedin.com/company/viratherapeutics-gmbh"
]
global_profiles = [u.rstrip("/").strip() for u in global_profiles]

# 2) Normalize your DataFrame’s LinkedIn URLs the same way
df_listings["LinkedIn_norm"] = (
    df_listings["LinkedIn"]
    .astype(str)
    .str.rstrip("/")
    .str.strip()
    .where(df_listings["LinkedIn"].notna(), None)
)

# 3) Flag true if it’s in the allowlist
df_listings["linkedin_global_profile"] = df_listings["LinkedIn_norm"].isin(global_profiles)

# 4) Valid AT-local profiles
mask_at_local = df_listings["LinkedIn_norm"].str.startswith("https://at.linkedin.com/company/")

# 5) Whitelist = AT-local OR in global allowlist
mask_keep = mask_at_local | df_listings["linkedin_global_profile"]

# 6) Set all other LinkedIn entries to NaN
df_listings.loc[~mask_keep, "LinkedIn"] = pd.NA

# 7) (Optional) drop the helper column
df_listings = df_listings.drop(columns=["LinkedIn_norm"])

print(df_listings[["name_cleaned","LinkedIn","linkedin_global_profile"]].head(20))


            name_cleaned                                       LinkedIn  linkedin_global_profile
0                 Avdain                                           <NA>                    False
1           Popper Power    https://at.linkedin.com/company/popperpower                    False
2              Setter AI                                           <NA>                    False
3           SurveySensum  https://www.linkedin.com/company/surveysensum                     True
4                 Artypa                                           <NA>                    False
5               Podpally                                           <NA>                    False
6        Wedding Capsule                                           <NA>                    False
7   Hyrox Training Plans                                           <NA>                    False
8                 506.ai         https://at.linkedin.com/company/506-ai                    False
9          Get Worksheet      

In [18]:
import pandas as pd

# 1) Build a mask: LinkedIn is non-null AND does *not* contain “/company/”
mask = (
    df_listings["LinkedIn"].notna()
    & ~df_listings["LinkedIn"].str.contains(r"/company/", na=False)
)

# 2) Slice out those rows
bad_links = df_listings.loc[mask, ["name_cleaned", "LinkedIn"]]

# 3) Inspect
print(f"{len(bad_links)} entries with non-company LinkedIn URLs:")
print(bad_links.to_string(index=False))

0 entries with non-company LinkedIn URLs:
Empty DataFrame
Columns: [name_cleaned, LinkedIn]
Index: []


In [20]:
import pandas as pd
from urllib.parse import urlparse, unquote

def extract_linkedin_handle(url: str) -> str | None:
    """
    Given a LinkedIn company URL (AT‐local or your global ones),
    returns the handle/slug after /company/.
    """
    if pd.isna(url) or not url:
        return None
    # Strip query‐string & fragment
    p = urlparse(url)
    path = unquote(p.path)  # decode %XX escapes
    # Split on “/”, drop empty parts
    parts = [seg for seg in path.split("/") if seg]
    # Find “company” and return what comes next
    if "company" in parts:
        idx = parts.index("company")
        if idx + 1 < len(parts):
            return parts[idx + 1].lower()
    return None

# Apply it
df_listings["linkedin_handle"] = df_listings["LinkedIn"].apply(extract_linkedin_handle)

# Inspect to verify
print(
    df_listings.loc[
        df_listings["linkedin_handle"].notna(),
        ["name_cleaned", "LinkedIn", "linkedin_handle"]
    ].head(10)
)

    name_cleaned                                           LinkedIn         linkedin_handle
1   Popper Power        https://at.linkedin.com/company/popperpower             popperpower
3   SurveySensum      https://www.linkedin.com/company/surveysensum            surveysensum
8         506.ai             https://at.linkedin.com/company/506-ai                  506-ai
11         DAIKI           https://at.linkedin.com/company/daiki-ai                daiki-ai
13       WasItAI            https://at.linkedin.com/company/wasitai                 wasitai
17         digna          https://www.linkedin.com/company/dignaai/                 dignaai
18      IDCanopy           https://at.linkedin.com/company/idcanopy                idcanopy
25   WEME Global         https://at.linkedin.com/company/wemeglobal              wemeglobal
27    SimVenture  https://at.linkedin.com/company/simventure-gmb...         simventure-gmbh
28     Seven Bel  https://at.linkedin.com/company/seven-bel-soun...  seven-bel-s

## Instagram Link

In [25]:
import pandas as pd
from urllib.parse import urlparse

def clean_instagram(url):
    """
    Keep only links like https://www.instagram.com/<username>/ 
    (exactly one path segment); else return pd.NA.
    """
    if pd.isna(url) or not url:
        return pd.NA

    u = url.strip().rstrip("/")  # drop trailing slash for parsing
    if not u.startswith(("http://", "https://")):
        u = "https://" + u

    p = urlparse(u)
    parts = [seg for seg in p.path.split("/") if seg]
    if len(parts) == 1:
        # re-add trailing slash for consistency
        return f"{u}/"
    return pd.NA

# Apply in place (or assign to a new column if you prefer)
df_listings["Instagram"] = df_listings["Instagram"].apply(clean_instagram)

# Optional: show which ones got dropped
invalid = df_listings["Instagram"].isna() & df_listings["Instagram"].notnull()
print("Dropped these entries as invalid Instagram URLs:")
print(df_listings.loc[invalid, ["name_cleaned", "Instagram"]].to_string(index=False))

Dropped these entries as invalid Instagram URLs:
Empty DataFrame
Columns: [name_cleaned, Instagram]
Index: []


In [27]:
import pandas as pd
from urllib.parse import urlparse

def extract_instagram_handle(url):
    """
    Given a cleaned Instagram URL of the form
      https://www.instagram.com/<username>/
    return the <username> (lowercased). Otherwise None.
    """
    if pd.isna(url) or not url:
        return None
    p = urlparse(url)
    # split path, drop empty segments
    parts = [seg for seg in p.path.split("/") if seg]
    # first segment is the username
    return parts[0].lower() if parts else None

# Apply to your DataFrame
df_listings["instagram_handle"] = df_listings["Instagram"].apply(extract_instagram_handle)

# Optionally, add a presence flag
df_listings["has_instagram"] = df_listings["instagram_handle"].notnull()

# Inspect the results
print(df_listings[["name_cleaned", "Instagram", "instagram_handle", "has_instagram"]].head())


   name_cleaned                                Instagram instagram_handle  has_instagram
0        Avdain                                     <NA>             None          False
1  Popper Power   https://www.instagram.com/popperpower/      popperpower           True
2     Setter AI     https://www.instagram.com/trysetter/        trysetter           True
3  SurveySensum  https://www.instagram.com/surveysensum/     surveysensum           True
4        Artypa        https://www.instagram.com/artypa/           artypa           True


In [28]:
df_listings.drop(columns=["has_instagram"], inplace=True)

### X.COm

In [33]:
import pandas as pd

# 1) Identify X URLs containing "status"
mask_status = df_listings["X"].str.contains("status", case=False, na=False)

# 2) Set those entries to NaN
df_listings.loc[mask_status, "X"] = pd.NA

# 3) (Optional) Inspect how many were removed
removed_count = mask_status.sum()
print(f"Removed {removed_count} X links containing 'status'.")

# 4) Verify remaining X URLs
print(df_listings.loc[df_listings["X"].notna(), "X"].head())

Removed 120 X links containing 'status'.
2              https://x.com/trysetter
3           https://x.com/surveysensum
6        https://x.com/Wedding_Capsule
8         https://x.com/506ai_?lang=vi
9    https://x.com/vidyacademy?lang=sr
Name: X, dtype: object


In [35]:
import pandas as pd

# 1) Build a mask for X URLs containing “hashtag” or “search”
mask_bad = df_listings["X"].str.contains(r"(hashtag|search)", case=False, na=False)

# 2) Set those entries to NaN
df_listings.loc[mask_bad, "X"] = pd.NA

# 3) (Optional) Inspect how many were removed
removed_count = mask_bad.sum()
print(f"Removed {removed_count} X links containing 'hashtag' or 'search'.")

# 4) Verify remaining X URLs
print(df_listings.loc[df_listings["X"].notna(), "X"].head())

Removed 14 X links containing 'hashtag' or 'search'.
2              https://x.com/trysetter
3           https://x.com/surveysensum
6        https://x.com/Wedding_Capsule
8         https://x.com/506ai_?lang=vi
9    https://x.com/vidyacademy?lang=sr
Name: X, dtype: object


C:\Users\mihim\AppData\Local\Temp\ipykernel_2588\483887481.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_bad = df_listings["X"].str.contains(r"(hashtag|search)", case=False, na=False)


In [37]:
import pandas as pd
from urllib.parse import urlparse, unquote

def extract_x_handle(url):
    """
    Given a URL like:
      https://x.com/Username
      https://x.com/Username?lang=de
    returns the 'Username' (lowercased), or None if missing.
    """
    if pd.isna(url) or not url:
        return None
    u = url.strip()
    if not u.startswith(("http://", "https://")):
        u = "https://" + u
    p = urlparse(u)
    # unquote in case of %-encoding, strip leading/trailing slashes
    path = unquote(p.path).strip("/")
    # take only the first segment (before any additional slash)
    handle = path.split("/")[0] if path else None
    return handle.lower() if handle else None

# Apply to your DataFrame
df_listings["x_handle"] = df_listings["X"].apply(extract_x_handle)

In [39]:
df_listings[["name_cleaned", "X", "x_handle"]].head(5)

,name_cleaned,X,x_handle
0,Avdain,<NA>,None
1,Popper Power,<NA>,None
2,Setter AI,https://x.com/trysetter,trysetter
3,SurveySensum,https://x.com/surveysensum,surveysensum
4,Artypa,<NA>,None


In [42]:
df_listings.head()

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons,startup,success_count,total_funding_eur,acquisition_count,bankruptcy_count,non_success_count,linkedin_global_profile,linkedin_handle,instagram_handle,x_handle
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,<NA>,<NA>,<NA>,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,[],NaN,Avdain,0,0,0,0,0,False,None,None,None
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,<NA>,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l...",4,"['Insolvenz | Popper Power GmbH', 'Insolvenz |...","['Reports a risk event (insolvency).', 'Report...",Popper Power,0,0,0,4,0,False,popperpower,popperpower,None
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,<NA>,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...,0,[],NaN,Setter AI,0,0,0,0,0,False,None,trysetter,trysetter
3,SurveySensum,https://www.eu-startups.com/directory/surveyse...,https://www.eu-startups.com/wp-content/uploads...,https://www.surveysensum.com,vienna,Vienna,"customer feedback, customer experience",Software & Analytics,SurveySensum is a leading customer feedback pl...,2018,7,No funding announced yet,Active,('https://www.linkedin.com/company/SurveySensu...,8,9,7.0,8,6,9,7,203,10,0.20,0,0,"['customer feedback', 'CX', 'business goals']",B2B,True,https://surveysensum.com,True,839.6,NaN,SurveySensum,SurveySensum,https://www.instagram.com/surveysensum/,https://x.com/surveysensum,https://www.linkedin.com/company/surveysensum,https://www.surveysensum.com/,https://surveysensum.com,surveysensum.com,surveysensum.com,[],0,[],NaN,SurveySensum,0,0,0,0,0,True,surveysensum,surveysensum,surveysensum
4,Artypa,https://www.eu-startups.com/directory/artypa/,https://www.eu-startups.com/wp-content/uploads...,https://artypa.com,Vienna,Vienna,"ai content, ai productivity, ai generation",Software & Analytics,Traditional AI workflows often involve navigat...,2024,1,No funding announced yet,Active,(),8,9,7.0,

## Verify User handles via Open AI API

by passing name_cleaned and top_3_keywords

In [43]:
df_listings[["name_cleaned", "linkedin_handle","instagram_handle", "x_handle"]].head(5)

,name_cleaned,linkedin_handle,instagram_handle,x_handle
0,Avdain,None,None,None
1,Popper Power,popperpower,popperpower,None
2,Setter AI,None,trysetter,trysetter
3,SurveySensum,surveysensum,surveysensum,surveysensum
4,Artypa,None,artypa,None


In [44]:
import os
import json
import pandas as pd
import openai
from dotenv import load_dotenv
from tqdm import tqdm

# 0) Bootstrap
load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [65]:
df_validation = df_listings

In [66]:
# ─── 1) Batch validation function spec ─────────────────────────────────────
batch_validate_fn = {
    "name": "validate_all_handles",
    "description": (
        "For each candidate handle, decide if it likely belongs to the given startup. "
        "Be permissive—only reject when clearly unrelated."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "startup": {
                "type": "string",
                "description": "Canonical startup name"
            },
            "results": {
                "type": "array",
                "description": "One entry per handle with verdict and reason",
                "items": {
                    "type": "object",
                    "properties": {
                        "platform":    {"type": "string"},
                        "handle":      {"type": "string"},
                        "valid":       {"type": "boolean"},
                        "reason":      {"type": "string"}
                    },
                    "required": ["platform","handle","valid"]
                }
            }
        },
        "required": ["startup","results"]
    }
}

# ─── 2) System prompt – more permissive rubric ───────────────────────────────
system_msg = {
    "role": "system",
    "content": (
        "You are a friendly verifier of social‐media handles. For each handle, "
        "answer True if it could reasonably be the startup’s official account, "
        "and False only if it’s clearly something else (wrong name, personal blog, "
        "generic page). Explain briefly why."
    )
}

In [67]:
# ─── 3) Helper to call the batch validator ──────────────────────────────────
def validate_all_handles(startup: str, candidates: list[dict], keywords: list[str]):
    """
    candidates: list of {"platform":..., "handle":...}
    returns: same list with valid+reason added
    """
    args = {"startup": startup, "results": candidates}
    prompt_lines = [f"- {c['platform']}: {c['handle']}" for c in candidates]
    user_msg = f"Startup: {startup}\nTop keywords: {', '.join(keywords)}\nHandles:\n" + "\n".join(prompt_lines)
    
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            system_msg,
            {"role":"user","content":user_msg}
        ],
        functions=[batch_validate_fn],
        function_call={
            "name": "validate_all_handles",
            "arguments": json.dumps(args)
        }
    )
    out = json.loads(resp.choices[0].message.function_call.arguments)
    return out["results"]

In [68]:
# ─── 4) Run over your df_listings ────────────────────────────────────────────
results = []
for _, row in tqdm(df_validation.iterrows(), total=len(df_validation), desc="Validating batches"):
    startup = row["name_cleaned"]
    keywords = row.get("top_3_keywords", []) or []
    candidates = []
    for plat, col in [("linkedin","linkedin_handle"),
                      ("instagram","instagram_handle"),
                      ("x","x_handle")]:
        h = row.get(col)
        if h:
            candidates.append({"platform": plat, "handle": h})
        else:
            # still include a placeholder to get a False+reason
            candidates.append({"platform": plat, "handle": ""})
    # one API call per startup
    verdicts = validate_all_handles(startup, candidates, keywords)
    for v in verdicts:
        results.append({
            "startup":  startup,
            "platform": v["platform"],
            "handle":   v["handle"],
            "valid":    v["valid"],
            "reason":   v.get("reason","")
        })

validations = pd.DataFrame(results)

Validating batches:  86%|████████▌ | 441/515 [15:45<02:38,  2.14s/it]


KeyError: 'valid'

In [70]:
validations = pd.DataFrame(results)

In [72]:
validation_backup = validations

In [78]:
validations.head(5)

,startup,platform,handle,valid,reason
0,Avdain,linkedin,,False,"Empty handle, does not represent an account."
1,Avdain,instagram,,False,"Empty handle, does not represent an account."
2,Avdain,x,,False,"Empty handle, does not represent an account."
3,Popper Power,linkedin,popperpower,True,"The handle matches the startup's name exactly,..."
4,Popper Power,instagram,popperpower,True,"The handle matches the startup's name exactly,..."


In [73]:
# 1) Suppose `validations` already exists with a column "startup"
processed = set(validations["startup"].dropna().unique())

# 2) Filter your original df (df_validation) to only those not in `processed`
remaining = df_validation.loc[
    ~df_validation["name_cleaned"].isin(processed)
].reset_index(drop=True)

print(f"{len(remaining)} startups left to validate out of {len(df_validation)}")

# 3) Now only loop over `remaining`
new_results = []
for _, row in tqdm(remaining.iterrows(),
                   total=len(remaining),
                   desc="Validating remaining"):
    startup  = row["name_cleaned"]
    keywords = row.get("top_3_keywords", []) or []
    candidates = []
    for plat, col in [("linkedin","linkedin_handle"),
                      ("instagram","instagram_handle"),
                      ("x","x_handle")]:
        h = row.get(col) or ""
        candidates.append({"platform":plat, "handle":h})

    try:
        verdicts = validate_all_handles(startup, candidates, keywords)
    except Exception as e:
        # If there's an error, mark all as invalid
        for c in candidates:
            new_results.append({
                "startup":  startup,
                "platform": c["platform"],
                "handle":   c["handle"],
                "valid":    False,
                "reason":   f"error: {e}"
            })
        continue

    for v in verdicts:
        new_results.append({
            "startup":  startup,
            "platform": v.get("platform",""),
            "handle":   v.get("handle",""),
            "valid":    v.get("valid", False),
            "reason":   v.get("reason","")
        })

74 startups left to validate out of 515


Validating remaining: 100%|██████████| 74/74 [02:39<00:00,  2.16s/it]


In [74]:
validations = pd.concat([validations, pd.DataFrame(new_results)], ignore_index=True)

In [79]:
validations.head(5)

,startup,platform,handle,valid,reason
0,Avdain,linkedin,,False,"Empty handle, does not represent an account."
1,Avdain,instagram,,False,"Empty handle, does not represent an account."
2,Avdain,x,,False,"Empty handle, does not represent an account."
3,Popper Power,linkedin,popperpower,True,"The handle matches the startup's name exactly,..."
4,Popper Power,instagram,popperpower,True,"The handle matches the startup's name exactly,..."


In [76]:
# 1) pivot out your results
val = validations.pivot_table(
    index="startup",
    columns="platform",
    values=["valid","reason"],
    aggfunc="first"
)

# 2) flatten the column MultiIndex
val.columns = [f"{metric}_{platform}" for metric,platform in val.columns]

# 3) merge back into df_validation
df_validation = df_validation.merge(
    val,
    left_on="name_cleaned",
    right_index=True,
    how="left"
)

In [80]:
df_validation.head(3)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons,startup,success_count,total_funding_eur,acquisition_count,bankruptcy_count,non_success_count,linkedin_global_profile,linkedin_handle,instagram_handle,x_handle,reason_instagram,reason_linkedin,reason_x,valid_instagram,valid_linkedin,valid_x
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,<NA>,<NA>,<NA>,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,[],NaN,Avdain,0,0,0,0,0,False,None,None,None,"Empty handle, does not represent an account.","Empty handle, does not represent an account.","Empty handle, does not represent an account.",False,False,False
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,<NA>,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l...",4,"['Insolvenz | Popper Power GmbH', 'Insolvenz |...","['Reports a risk event (insolvency).', 'Report...",Popper Power,0,0,0,4,0,False,popperpower,popperpower,None,"The handle matches the startup's name exactly,...","The handle matches the startup's name exactly,...","The handle is missing, which cannot represent ...",True,True,False
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,<NA>,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...,0,[],NaN,Setter AI,0,0,0,0,0,False,None,trysetter,trysetter,"The handle 'trysetter' is short and relevant, ...",The handle is missing.,"Similar to Instagram, 'trysetter' is a suitabl...",True,False,True


In [81]:
df_validation.to_csv("./data/eustartup_listings_enriched_5.csv", index=False)